In [11]:
from owlready2 import *

In [12]:
onto = get_ontology("BreastCancerTreatment.rdf").load()

In [3]:
print(onto)

get_ontology("http://www.semanticweb.org/kaixuankhoo/ontologies/2022/6/BreastCancerTreatments#")


In [12]:
sync_reasoner_pellet()

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\antlr-3.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;c:\Users\Kaixua

In [4]:
print(Thing)

owl.Thing


In [10]:
# print(onto)
print(onto.Patient1.PatientID)

AttributeError: 'NoneType' object has no attribute 'PatientID'

## Playing around with Querying

In [6]:
## searching using IRI name
onto.search(iri = "*REF")

[untitled-ontology-24.Capecitabine-REF, untitled-ontology-24.Carboplatin-REF, untitled-ontology-24.Cyclophosphamide-REF, untitled-ontology-24.Docetaxel-REF, untitled-ontology-24.Epirubicin-REF, untitled-ontology-24.Infusion-REF, untitled-ontology-24.Intramuscular-REF, untitled-ontology-24.Intravenous-REF, untitled-ontology-24.Paclitaxel-REF, untitled-ontology-24.Pertuzumab-REF, untitled-ontology-24.Subcutaneous-REF, untitled-ontology-24.Trastuzumab-REF, untitled-ontology-24.5Fluorouracil-REF]

In [16]:
## searching relationships using object properties
onto.search(has_tumour = "*")

[untitled-ontology-24.Patient1]

In [55]:
## searching subclass of something
onto.search(subclass_of = onto.Drug_reference)

[BreastTreatment2.Drug_reference, BreastTreatment2.AlkylatingAgent, BreastTreatment2.Anthracycline, BreastTreatment2.AntiMetabolite, BreastTreatment2.Taxane, BreastTreatment2.HER2Targeted, BreastTreatment2.PlatinumBased, BreastTreatment2.NitrogenMustardBased, BreastTreatment2.Epirubicin, BreastTreatment2.Capecitabine, BreastTreatment2.5-FU, BreastTreatment2.Docetaxel, BreastTreatment2.Paclitaxel, BreastTreatment2.Pertuzumab, BreastTreatment2.Trastuzumab, BreastTreatment2.Carboplatin, BreastTreatment2.Cyclophosphamide]

In [9]:
onto.Tumour